In [3]:
import os

# Define the path to the folder containing your Verilog files
folder_path = 'encoded'

# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


405
<class 'str'>
encoded\adder11.txt


In [4]:
import os
import json
import re
import shutil

def label_verilog_file(file_name):
    label_mapping = {
        'adder': 0, 'ALU': 1, 'and': 2, 'nand': 3, 'nor': 4, 'not': 5, 'or': 6,'xnor': 7, 'xor': 8,'comparator': 9, 'decoder': 10,
        'encoder': 11, 'mult': 12, 'mux': 13, 'pe': 14, 'seg': 15, 'sub': 16
    }
    pattern = r"([a-zA-Z]+)(\d+)?"
    match = re.match(pattern, file_name)
    if match:
        base_name = match.group(1)
        if base_name in label_mapping:
            return label_mapping[base_name]
        
    return None

def add_label_to_verilog_file(input_file_path, output_folder):
    if input_file_path.endswith('.txt'):
        with open(input_file_path, "r") as file:
            loaded_data = json.load(file)
            # Convert string elements to integers
            for sublist in loaded_data[0]:
                sublist[1] = int(sublist[1])
            label = label_verilog_file(os.path.basename(input_file_path))
            if label is not None and [label] not in loaded_data:  # Check if label already exists
                loaded_data.append([label])  # Add label as the third list
                output_file_path = os.path.join(output_folder, os.path.basename(input_file_path))
                with open(output_file_path, "w") as output_file:
                    json.dump(loaded_data, output_file)
                return True
    return False

input_folder = "encoded"
output_folder = "done"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process files in the input folder
for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)
    if os.path.isfile(file_path):
        if not add_label_to_verilog_file(file_path, output_folder):
            print(f"Failed to label: {file_path}")

# Optionally, delete the input files after processing
# for file_name in os.listdir(input_folder):
#     file_path = os.path.join(input_folder, file_name)
#     if os.path.isfile(file_path):
#         os.remove(file_path)
